Mengisi nilai variabel KAGGLE_USERNAME dan KAGGLE KEY agar dapat mendownload dataset langsung ke colab

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = 'arfalrazya'
os.environ['KAGGLE_KEY'] = '06d59d4ccf72432919159fdb46ba5caf'

Mendownload datasets dengan API yang telah disediakan di halaman dataset

In [3]:
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

anime-recommendation-database-2020.zip: Skipping, found more recently modified local copy (use --force to force download)


Melakukan Ekstraksi datasets dengan command unzip

In [4]:
!unzip 'anime-recommendation-database-2020.zip'

'unzip' is not recognized as an internal or external command,
operable program or batch file.


#Data Loading
Membaca dataset pada file **anime_with_synopsis.csv**. Ini dilakukan menggunakan fungsi **read_csv** dari library pandas





In [1]:
import pandas as pd
 
anime_synopsis = pd.read_csv('./anime-recommendation-database-2020/anime_with_synopsis.csv')

#Data Understandings

## Melihat informasi dari datasets

Melihat informasi kolom pada dataset anime_synopsis dengan fungsi **info()**. Hasil dari fungsi info memberikan infromasi bahwa terdapat satu tipe data integer pada kolom MAL_ID dan tipe data object untuk kolom lainnya.

In [2]:
anime_synopsis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAL_ID     16214 non-null  int64 
 1   Name       16214 non-null  object
 2   Score      16214 non-null  object
 3   Genres     16214 non-null  object
 4   sypnopsis  16206 non-null  object
dtypes: int64(1), object(4)
memory usage: 633.5+ KB


## Melihat Baris pada datasets

Melihat 5 baris teratas dari datasets anime_synopsis. dengan fungsi **head()**. Fungsi tersebut akan menghasilkan 5 baris teratas dari datasets anime_synopsis beserta nilai pada setiap kolom pada baris tersebut.

In [3]:
anime_synopsis.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


#Data Preparation
##Pengecekan Missing Value

Melakukan pengecekan untuk missing value menggunakan fungsi **isnull()**, kemudian menjumlhkan nilai pada kolom yang  mengandung missing value. Dari Hasil pengecekan pada datasets **anime_synopsis** terdapat 8 missing value pada kolom synopsis.

In [4]:
anime_synopsis.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64

## *Missing Value Handling*

Mengganti missing value pada kolom sinopsis dengan nilai string kosong. Hai ini dilakukan agar setiap baris pada kolom synopsis nantinya dapat direpresentasikan dengan TF-IDF. Ini dilakukan dengan fungsi **fillna()** yang akan mengisi nilai yang kosong pada kolom synopsis dengan string kosong.

In [5]:
anime_synopsis['sypnopsis'] = anime_synopsis['sypnopsis'].fillna('')

#Model Development dengan Content Based Filtering
## TF-IDF Vectorizer
Melakukan tokenisasi dengan **TfidfVectorizer()** dari library sklearn dengan masukan kolom synopsis dari datasets anime_synopsis. Tokenisasi dilakukan untuk menemukan representasi fitur penting dari setiap sinopsis.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tfidf.fit_transform(anime_synopsis['sypnopsis'])
tfidf_matrix.shape

(16214, 45363)

In [17]:
tfidf_matrix

<16214x45363 sparse matrix of type '<class 'numpy.float64'>'
	with 748499 stored elements in Compressed Sparse Row format>

## Membuat matriks TFIDF
Membuat hasil tf-idf dalam bentuk matriks, menggunakan fungsi **todense()**. Ini dilakukan agar dapat menghitung kesamaan hasil tokenisasi dengan fungsi cosine similarity 

In [7]:
tfidf_matrix.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.23540208, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

##Similarity Measures
Menghitung derajat kesamaan (similarity degree) antar sinopsis dengan teknik **cosine similarity**. Menggunakan fungsi cosine_similarity dari library sklearn.Perhitungan **cosine similarity** dataframe tfidf_matrix yang kita peroleh pada tahapan sebelumnya. 

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(tfidf_matrix) 
similarity

# from sklearn.metrics.pairwise import euclidean_distances

# similarity = euclidean_distances(tfidf_matrix) 
# similarity

array([[1.        , 0.26802724, 0.0995299 , ..., 0.00454389, 0.00888682,
        0.07977393],
       [0.26802724, 1.        , 0.10013954, ..., 0.00518709, 0.        ,
        0.05853303],
       [0.0995299 , 0.10013954, 1.        , ..., 0.00861021, 0.        ,
        0.04168639],
       ...,
       [0.00454389, 0.00518709, 0.00861021, ..., 1.        , 0.02999185,
        0.00518434],
       [0.00888682, 0.        , 0.        , ..., 0.02999185, 1.        ,
        0.01520908],
       [0.07977393, 0.05853303, 0.04168639, ..., 0.00518434, 0.01520908,
        1.        ]])

## Matriks Hasil perhitungan Cosine Similarity

Melihat matriks kesamaan setiap sinopsis anime dengan menampilkan nama anime dalam 5 sampel kolom (axis = 1) dan 10 sampel baris (axis=0). Dari matriks ini dapat dilihat nilai yang dihasil kan untuk setiap element bernilai 0 sampai 1. Sebagai contoh pada element [0,0] nilai kesamaan anime "Dai Yamato Zero-gou" dengan "Niehime to Kemono no Ou" memiliki nilai kesamaan 0.078287	

In [9]:
similarity_df = pd.DataFrame(similarity, index=anime_synopsis['Name'], columns=anime_synopsis['Name'])
print('Shape:', similarity_df.shape)
 
# Melihat similarity matrix pada setiap resto
similarity_df.sample(5, axis=1).sample(10, axis=0)

Shape: (16214, 16214)


Name,Pac-World,Dai Mahou Touge Omake,Chou Denji Machine Voltes V,Ganbare Swimmy,City Hunter: Hyakuman Dollar no Inbou
Name,,,,,
Kitty-chan wo Sagase! Toyama-shi Stamp Rally,0.000000,0.000000,0.000000,0.000000,0.000000
Detective Conan: Black History 2,0.000000,0.000000,0.004055,0.000000,0.006607
Straw-saurus Neo,0.041534,0.004169,0.064206,0.030023,0.018823
Na mo Nai Hana no Youni,0.002578,0.008433,0.057748,0.004637,0.026862
Soukyuu no Fafner: Dead Aggressor - The Beyond,0.004990,0.000000,0.048630,0.013153,0.003314
Master of Torque 3,0.020968,0.013134,0.036709,0.025520,0.003839
You Should See Me in a Crown,0.018430,0.000000,0.035768,0.016667,0.031070
Pokemon Movie 03: Kesshoutou no Teiou Entei,0.052893,0.000000,0.109172,0.036594,0.040071
Hetalia: The World Twinkle Extra Disc,0.032963,0.002563,0.061625,0.023464,0.039801


In [11]:
similarity_df.to_pickle("./dummy.pkl")

##Pembentukan Fungsi Rekomendasi

Membuat fungsi anime_recommendations dengan beberapa parameter:
  **Anime** : Nama anime(index kemiripan dataframe).
-   **Similarity_data** : Dataframe mengenai similarity yang telah kita definisikan sebelumnya.
-   **items** : Nama dan fitur yang digunakan untuk mendefinisikan kemiripan. Dalam hal ini **Name** yang merujuk pada nama anime dan **synopsis** yang merupakan sinopsis dari nama anime yang diberikan.
-   **k** : Banyak rekomendasi yang ingin diberikan.

Fungsi ini akan menghasilkan keluaran berupa Top N rekomendasi, yang nilai N akan disesuaikan pada parameter k) berdasarkan kesamaan pada matriks similarity. Keluaran yang dihasilkan adalah judul anime, sinopsis dan genre.



In [14]:
def anime_recommendations(anime, similarity_data=similarity_df, items=anime_synopsis[['Name', 'sypnopsis','Genres']], k=5):

    index = similarity_data.loc[:,anime].to_numpy().argpartition(
        range(-1, -k, -1))
    
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    closest = closest.drop(anime, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

#Evaluasi
##Melakukan Pengecekan Data
Hal pertama untuk melakukan evaluasi, disini dilihat terlebih dahulu padakah judul anime yang akan dimasukkan ke fungsi rekomendasi ada dalam datasets. Pengecekan dilakukan dengan bantuan fungsi **eq()**. Dalam kasus ini akan dipastikan anime dengan judul One Punch Man ada dalam datasets.

In [15]:
anime_synopsis[anime_synopsis.Name.eq('One Punch Man')]

,MAL_ID,Name,Score,Genres,sypnopsis
9394,30276,One Punch Man,8.57,"Action, Sci-Fi, Comedy, Parody, Super Power, S...",The seemingly ordinary and unimpressive Saitam...


##Uji Coba Fungsi
Pada pengujian ini akan dilakukan untuk mendapatkan **Top 5 Recommendation**.
Judul Anime yang telah diperiksa tadi dimasukkan dalam fungsi anime **anime_recommendations()**. Dapat dilihat hasil dari fungsi tersebut menghasilkan 5 buah rekomendasi sesuai dengan parameter k.

In [16]:
anime_recommendations('One Punch Man')

,Name,sypnopsis,Genres
0,One Punch Man: Road to Hero,"Before Saitama became the man he is today, he ...","Action, Sci-Fi, Comedy, Parody, Super Power, S..."
1,Aura: Maryuuin Kouga Saigo no Tatakai,Ichirou Satou is an ordinary high school stude...,"Supernatural, Drama, Romance, School"
2,One Punch Man 2nd Season,In the wake of defeating Boros and his mighty ...,"Action, Sci-Fi, Comedy, Parody, Super Power, S..."
3,Yumeria,"On his 16th birthday, Tomokazu Mikuri had a re...","Action, Comedy, Ecchi, Harem, Super Power"
4,Dog Days',Cinque returns to the land of Flonyard in Dog ...,"Action, Adventure, Magic, Fantasy"
